In [1]:
# !pip install -qU langchain-graph-retriever
# !pip install "langchain-graph-retriever[chroma]"
# !pip install -qU langchain-community pypdf

In [2]:
import os

In [4]:
# Read documents from directory
from langchain_community.document_loaders import PyPDFLoader

file_path = "../docs/Personal Data English V2-23April2023- Reviewed-.pdf"
loader = PyPDFLoader(file_path)


In [5]:
docs = loader.load()
docs[0]

Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 36 0 (offset 0)
Ignoring wrong pointing object 57 0 (offset 0)


Document(metadata={'producer': 'macOS Version 13.1 (Build 22C65) Quartz PDFContext', 'creator': 'Word', 'creationdate': "D:20230423214602Z00'00'", 'title': 'Microsoft Word - Personal Data English V2-23April2023- Reviewed-.docx', 'moddate': "D:20230423214602Z00'00'", 'source': '../docs/Personal Data English V2-23April2023- Reviewed-.pdf', 'total_pages': 16, 'page': 0, 'page_label': '1'}, page_content='')

In [6]:
import pprint
pprint.pp(docs[0].metadata)

{'producer': 'macOS Version 13.1 (Build 22C65) Quartz PDFContext',
 'creator': 'Word',
 'creationdate': "D:20230423214602Z00'00'",
 'title': 'Microsoft Word - Personal Data English V2-23April2023- '
          'Reviewed-.docx',
 'moddate': "D:20230423214602Z00'00'",
 'source': '../docs/Personal Data English V2-23April2023- Reviewed-.pdf',
 'total_pages': 16,
 'page': 0,
 'page_label': '1'}


## Vectore Store Definition

In [ ]:
from langchain_chroma.vectorstores import Chroma
from langchain_graph_retriever.transformers import ShreddingTransformer

vector_store = Chroma.from_documents(
    # documents=list(ShreddingTransformer().transform_documents(animals)),
    # embedding=embeddings,
    # collection_name="animals",
)

### Experimeting Chunking Strategy

*Page -> Article -> Paragraph*

**Metdata** 
- Regulation Name (same for all)
- Article Number
- Paragraph ID
- Version Date (same for all)
- Language (English, same for all)
- Sector: keyword mapping table + scoring — produce list of matched sectors per article (e.g. `["government","health","finance"]`) and a primary sector chosen by strongest match.